In [2]:
from Graph import *
from collections import Counter
import pickle
from simplification import *
g = pickle.load(open("random_graph_3", "rb"))
print(len(g.vertices))
print(len(g.edges))

50
84


In [ ]:
g = simplify_double_edges(g)
print(len(g.vertices), len(g.edges))
g = delete_chain_vertices(g)
print(len(g.vertices), len(g.edges))

Links drawing attempt:

In [3]:
g.are_connected(list(g.vertices)[0], list(g.vertices)[5])

False

In [ ]:
def link(g: Graph v1: Vertex, v2: Vertex) -> list:
    '''not tested yet
    1) run it on edges
    2) return the vertices as well'''
    
    return_list = []
    
    # if v1 not linked to v2 - return some shit
    if not g.are_connected(v1, v2):
        return "impossible to draw a link between unconnected vertices!"
    
    # find all vertices v1 links to (besides v2) - can be sped up most likely
    vertices_v1_links_to = []
    for edge in v1.edges:
        for vertex in edge.vertices:
            if vertex not in [v1, v2]:
                vertices_v1_links_to.append(vertex)
    
    # among those, leave only those vertices that are connected to v2
    vertices_v1_links_to = [vertex for vertex in vertices_v1_links_to if g.are_connected(vertex, v2)]
    
    # for each vertex in that list:
    for vertex in vertices_v1_links_to:
    
    # for each edge belonging to that vertex:
        for edge in vertex.edges:
    
    # if edge connects to v1 or v2, add it to a return list
            if v1 in edge.vertices or v1 in edge.vertices:
                return_list.append(edge)
    
    return return_list

Old approach:

In [ ]:
# def simplify_double_edges(g: Graph):
#     '''if there are 2 edges within a pair of vertices, leave only 1
#     This is possible due to perhaps multiple edge objects (of different IDs) connecting the same Vertices
    
#     Also fixes broken edges'''
    
#     unique_vertex_connections = []
#     edges_copy = g.edges.copy()

#     for edge in edges_copy:
        
#         #Not broken edge (2 endpoints)
#         try:
#             pair = (list(edge.vertices)[0].id, list(edge.vertices)[1].id)
            
#             if pair not in unique_vertex_connections:
#                 unique_vertex_connections.append(pair)
#             else:
#                 print("removed edge via try")
# #                 list(edge.vertices)[0].remove_edges(edge) #vertex.remove_edges(edge)
# #                 list(edge.vertices)[1].remove_edges(edge)
#                 g.edges.remove(edge) #remember to update properly
        
#         #Broken edge (1 endpoint)
#         except IndexError:
#             print('removed an edge via except')
#             #list(edge.vertices)[0].remove_edges(edge)
#             g.edges.remove(edge)
            
#     g.edges = edges_copy #?
    
#     return g